# library

In [1]:
# Import necessary libraries
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import ResNet50, ResNet152, VGG16
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, Average
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

import random
import time
opt = Adam(learning_rate = 0.0001)
model_path = './model_224_32_50'

In [2]:
tf.test.is_gpu_available()

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


True

In [3]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

# Data Preprocess

In [ ]:
# Set random seed for reproducibility
np.random.seed(42)
tf.random.set_seed(42)


# Set the image size, batch size, and number of classes
img_width, img_height = 224, 224
batch_size = 32
num_classes = 2


# Define the paths to your training and validation data directories
data_dir = './data_original'
#train_data = './Brain Tumor/Training - Full'
#test_data = './Brain Tumor/Testing - Full'

# Define the data generators for training and validation
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)


train_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

valid_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

In [78]:
# Get the class names
class_names = list(train_generator.class_indices.keys())

In [79]:
class_names

['no_tumor', 'pituitary']

# ResNet 152

In [80]:
# Load the pre-trained ResNet50 model without the top (fully connected) layers
base_model = ResNet152(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))

In [81]:
# Freeze the pre-trained layers
for layer in base_model.layers:
    layer.trainable = False

x = base_model.output
x = tf.keras.layers.GlobalAveragePooling2D()(x)
#x = tf.keras.layers.Dense(512, activation='relu')(x)
x = tf.keras.layers.Dense(128, activation='relu')(x)
x = tf.keras.layers.Dense(64, activation='relu')(x)
x = tf.keras.layers.Dropout(0.5)(x)
#predictions = Dense(2, activation = 'softmax')(x)
predictions = tf.keras.layers.Dense(2,activation='sigmoid')(x)
#dense_model = Model(inputs = base_model.inputs, outputs = predictions)

model_resnet = tf.keras.models.Model(inputs=base_model.input, outputs=predictions)

In [82]:
# Compile the model
#model_resnet.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model_resnet.compile(optimizer=opt, loss=tf.keras.losses.BinaryCrossentropy(), metrics=['accuracy'])

In [83]:
#regularization: EARLY STOPPING
#early_stopping = EarlyStopping(monitor='val_loss', patience=5, verbose=1)

In [ ]:
# Train the model
start_time = time.time()

history_resnet = model_resnet.fit(train_generator,
          steps_per_epoch=train_generator.n // batch_size,
          epochs=50,
          validation_data=valid_generator ,
          #callbacks=[early_stopping]
          validation_steps=valid_generator.n // batch_size
        )

In [85]:
end_time = time.time()
training_time = end_time - start_time
print(f"Training time fro ResNet 152: {training_time} seconds")

Training time fro ResNet 152: 1996.7869460582733 seconds


In [86]:
model_resnet.save(model_path + '/resnet152.h5')

In [ ]:
# load model
model_resnet = load_model(model_path + '/resnet152.h5')
model_resnet.summary()

# Model Evaluation Metrics
[link](https://machinelearningmastery.com/how-to-calculate-precision-recall-f1-and-more-for-deep-learning-models/)

In [ ]:
# Evaluate the model
valid_generator.reset()
#predictions = model.predict(valid_generator, steps=valid_generator.n // batch_size)
start_time = time.time()
y_pred = model_resnet.predict(valid_generator)   #yhat_probs

end_time = time.time()

inference_time_batch = end_time - start_time
print("Inference time:", inference_time_batch)

# Get the predicted classes
predicted_classes = np.argmax(y_pred, axis=1)    #yhat_classes


# Get the true classes
true_classes = valid_generator.classes


# Calculate performance metrics
confusion = confusion_matrix(true_classes, predicted_classes)
tn, fp, fn, tp = confusion.ravel()
print("tn: ", tn ,"fp: ", fp,"fn: ", fn,"tp: ", tp)
specificity = tn / (tn + fp)
precision = precision_score(true_classes, predicted_classes)
recall = recall_score(true_classes, predicted_classes)
f1 = f1_score(true_classes, predicted_classes)

# Print the performance metrics
print('Specificity:', specificity)
print('Precision:', precision)
print('Recall:', recall)
print('F1 Score:', f1)

Accuracy =  (tp + tn) / ( tp + tn + fp + fn)
print("Accuracy: " , Accuracy)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(true_classes, predicted_classes))

In [ ]:
import seaborn as sns
cm = confusion_matrix(true_classes, predicted_classes)
#sns.heatmap(cm, annot=True, cmap='Blues')
cm_data = [[248, 42], [30, 297]]
sns.heatmap(cm_data, annot=True, cmap='Blues', fmt='d', xticklabels=class_names, yticklabels=class_names)

# Graphs, Matrix

**331, batch 32**

Specificity: 0.0
Precision: 0.5688816855753647
Recall: 1.0
F1 Score: 0.7252066115702479

**331, batch 64**
tn:  96 fp:  170 fn:  129 tp:  222
Specificity: 0.3609022556390977
Precision: 0.5663265306122449
Recall: 0.6324786324786325
F1 Score: 0.5975773889636609

**331, batch 64, RELU**

tn:  266 fp:  0 fn:  351 tp:  0
Specificity: 1.0
Precision: 0.0
Recall: 0.0
F1 Score: 0.0

In [ ]:
loss,acc = model_resnet.evaluate(valid_generator)

In [ ]:
# Plot the confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(confusion, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('ResNet 152 Confusion Matrix')
plt.savefig(model_path + '/resnet_conf_matrix.png')
plt.show()

In [ ]:
plt.figure(figsize=(10, 4))
plt.subplot(1, 2, 1)
plt.plot(history_resnet.history['loss'], label='Training Loss')
plt.plot(history_resnet.history['val_loss'], label='Validation Loss')
plt.title('ResNet-152')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()


plt.xticks([10, 20, 30])
plt.yticks([.2, .4, .6, .8, 1, 1.2])
plt.savefig(model_path + '/resnet_loss.png')


# Plot training accuracy and validation accuracy
plt.subplot(1, 2, 2)
plt.plot(history_resnet.history['accuracy'], label='Training Accuracy')
plt.plot(history_resnet.history['val_accuracy'], label='Validation Accuracy')
plt.title('ResNet-152')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.xticks([10, 20, 30])
plt.yticks([.2, .4, .6, .8, 1, 1.2])
plt.savefig(model_path + '/resnet_acc.png')
# Adjust layout and display the plot
plt.tight_layout()
plt.show()


# DenseNet 201

In [88]:
# Load the pre-trained ResNet50 model without the top (fully connected) layers
base_model = tf.keras.applications.DenseNet201(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))

In [89]:
# Freeze the pre-trained layers
for layer in base_model.layers:
    layer.trainable = False

# Add custom fully connected layers for classification
x = base_model.output
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dense(128, activation='relu')(x)
x = tf.keras.layers.Dense(64, activation='relu')(x)
predictions = tf.keras.layers.Dense(num_classes, activation='sigmoid')(x)

# Create the final model
model_dense = tf.keras.models.Model(inputs=base_model.input, outputs=predictions)

In [90]:
# Compile the model
model_dense.compile(optimizer=opt, loss=tf.keras.losses.BinaryCrossentropy(), metrics=['accuracy'])

In [ ]:
# Train the model
start_time = time.time()

history_model_dense = model_dense.fit(train_generator,
          steps_per_epoch=train_generator.n // batch_size,
          epochs=50,
          validation_data=valid_generator ,
          validation_steps=valid_generator.n // batch_size
                                      )

In [ ]:
end_time = time.time()
training_time = end_time - start_time
print(f"Training time for DenseNet 201: {training_time} seconds")

In [93]:
model_dense.save(model_path + '/dense201.h5')

# Model Evaluation Metrics for DenseNet


[link](https://machinelearningmastery.com/how-to-calculate-precision-recall-f1-and-more-for-deep-learning-models/)

In [ ]:
# Evaluate the model
valid_generator.reset()
#predictions = model.predict(valid_generator, steps=valid_generator.n // batch_size)
start_time = time.time()
predictions = model_dense.predict(valid_generator)   #yhat_probs

end_time = time.time()

inference_time_batch = end_time - start_time
print("Inference time:", inference_time_batch)

# Get the predicted classes
predicted_classes = np.argmax(predictions, axis=1)    #yhat_classes

# Get the true classes
true_classes = valid_generator.classes


# Calculate performance metrics
confusion = confusion_matrix(true_classes, predicted_classes)
tn, fp, fn, tp = confusion.ravel()
print("tn: ", tn ,"fp: ", fp,"fn: ", fn,"tp: ", tp)
specificity = tn / (tn + fp)
precision = precision_score(true_classes, predicted_classes)
recall = recall_score(true_classes, predicted_classes)
f1 = f1_score(true_classes, predicted_classes)

# Print the performance metrics
print('Specificity:', specificity)
print('Precision:', precision)
print('Recall:', recall)
print('F1 Score:', f1)

Accuracy =  (tp + tn) / ( tp + tn + fp + fn)
print("Accuracy: " , Accuracy)


In [ ]:
# Evaluate the model
valid_generator.reset()
#predictions = model.predict(valid_generator, steps=valid_generator.n // batch_size)
predictions = model_dense.predict(valid_generator)

In [29]:
# Get the predicted classes
predicted_classes = np.argmax(predictions, axis=1)

# Get the true classes
true_classes = valid_generator.classes

In [ ]:
# Calculate performance metrics
confusion = confusion_matrix(true_classes, predicted_classes)
tn, fp, fn, tp = confusion.ravel()
specificity = tn / (tn + fp)
precision = precision_score(true_classes, predicted_classes)
recall = recall_score(true_classes, predicted_classes)
f1 = f1_score(true_classes, predicted_classes)

# Print the performance metrics
print('Specificity:', specificity)
print('Precision:', precision)
print('Recall:', recall)
print('F1 Score:', f1)

In [31]:
Accuracy =  (tp + tn) / ( tp + tn + fp + fn)

In [ ]:
Accuracy

In [ ]:
loss,acc = model_dense.evaluate(valid_generator)

In [ ]:
# Plot the confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(confusion, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('DenseNet 201 Confusion Matrix')
plt.savefig(model_path + '/densenet_conf_matrix.png')
plt.show()

In [ ]:
plt.figure(figsize=(10, 4))
plt.subplot(1, 2, 1)
plt.plot(history_model_dense.history['loss'], label='Training Loss')
plt.plot(history_model_dense.history['val_loss'], label='Validation Loss')
plt.title('DenseNet-201')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.savefig(model_path + '/dense_loss.png')
plt.xticks([10, 20, 30])
plt.yticks([.2, .4, .6, .8, 1, 1.2])

# Plot training accuracy and validation accuracy
plt.subplot(1, 2, 2)
plt.plot(history_model_dense.history['accuracy'], label='Training Accuracy')
plt.plot(history_model_dense.history['val_accuracy'], label='Validation Accuracy')
plt.title('DenseNet-201')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.xticks([10, 20, 30])
plt.yticks([.2, .4, .6, .8, 1, 1.2])
plt.savefig(model_path + '/densenet_acc.png')
# Adjust layout and display the plot
plt.tight_layout()
plt.show()


# vgg 16

In [95]:
# Load the pre-trained ResNet50 model without the top (fully connected) layers
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))

In [96]:
# Freeze the pre-trained layers
for layer in base_model.layers:
    layer.trainable = False

# Add custom fully connected layers for classification
x = base_model.output
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dense(128, activation='relu')(x)
x = tf.keras.layers.Dense(64, activation='relu')(x)
predictions = tf.keras.layers.Dense(num_classes, activation='sigmoid')(x)

# Create the final model
model_vgg16 = tf.keras.models.Model(inputs=base_model.input, outputs=predictions)

In [97]:
# Compile the model
model_vgg16.compile(optimizer=opt, loss=tf.keras.losses.BinaryCrossentropy(), metrics=['accuracy'])

In [ ]:
# Train the model
start_time = time.time()

history_model_vgg16 = model_vgg16.fit(train_generator,
          steps_per_epoch=train_generator.n // batch_size,
          epochs=50,
          validation_data=valid_generator,
          validation_steps=valid_generator.n // batch_size)

In [ ]:
end_time = time.time()
training_time = end_time - start_time
print(f"Training time for VGG 16: {training_time} seconds")

In [100]:
model_vgg16.save(model_path + '/vgg16.h5')

In [ ]:
# Evaluate the model
valid_generator.reset()
#predictions = model.predict(valid_generator, steps=valid_generator.n // batch_size)
start_time = time.time()
predictions = model_vgg16.predict(valid_generator)

end_time = time.time()

inference_time_batch = end_time - start_time
print("Inference time:", inference_time_batch)

In [43]:
# Get the predicted classes
predicted_classes = np.argmax(predictions, axis=1)

# Get the true classes
true_classes = valid_generator.classes

In [ ]:
# Calculate performance metrics
confusion = confusion_matrix(true_classes, predicted_classes)
tn, fp, fn, tp = confusion.ravel()
specificity = tn / (tn + fp)
precision = precision_score(true_classes, predicted_classes)
recall = recall_score(true_classes, predicted_classes)
f1 = f1_score(true_classes, predicted_classes)

# Print the performance metrics
print('Specificity:', specificity)
print('Precision:', precision)
print('Recall:', recall)
print('F1 Score:', f1)

In [45]:
Accuracy =  (tp + tn) / ( tp + tn + fp + fn)

In [ ]:
Accuracy

In [ ]:
loss,acc = model_vgg16.evaluate(valid_generator)

In [ ]:
loss

In [ ]:
# Plot the confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(confusion, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('VGG 16 Confusion Matrix')
plt.savefig(model_path + '/vgg_conf_matrix.png')
plt.show()

In [ ]:
plt.figure(figsize=(10, 4))
plt.subplot(1, 2, 1)
plt.plot(history_model_vgg16.history['loss'], label='Training Loss')
plt.plot(history_model_vgg16.history['val_loss'], label='Validation Loss')
plt.title('VGG-16')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.xticks([10, 20, 30])
plt.yticks([.2, .4, .6, .8, 1, 1.2])
plt.savefig(model_path + '/vgg_loss.png')

# Plot training accuracy and validation accuracy
plt.subplot(1, 2, 2)
plt.plot(history_model_vgg16.history['accuracy'], label='Training Accuracy')
plt.plot(history_model_vgg16.history['val_accuracy'], label='Validation Accuracy')
plt.title('VGG-16')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.xticks([10, 20, 30])
plt.yticks([.2, .4, .6, .8, 1, 1.2])
plt.savefig(model_path + '/vgg_acc.png')
# Adjust layout and display the plot
plt.tight_layout()
plt.show()



# Model Evaluation Metrics for VGG
[link](https://machinelearningmastery.com/how-to-calculate-precision-recall-f1-and-more-for-deep-learning-models/)

In [ ]:
# Evaluate the model
valid_generator.reset()
#predictions = model.predict(valid_generator, steps=valid_generator.n // batch_size)
predictions = model_vgg16.predict(valid_generator)   #yhat_probs

# Get the predicted classes
predicted_classes = np.argmax(predictions, axis=1)    #yhat_classes

# Get the true classes
true_classes = valid_generator.classes


# Calculate performance metrics
confusion = confusion_matrix(true_classes, predicted_classes)
tn, fp, fn, tp = confusion.ravel()
print("tn: ", tn ,"fp: ", fp,"fn: ", fn,"tp: ", tp)
specificity = tn / (tn + fp)
precision = precision_score(true_classes, predicted_classes)
recall = recall_score(true_classes, predicted_classes)
f1 = f1_score(true_classes, predicted_classes)

# Print the performance metrics
print('Specificity:', specificity)
print('Precision:', precision)
print('Recall:', recall)
print('F1 Score:', f1)

Accuracy =  (tp + tn) / ( tp + tn + fp + fn)
print("Accuracy: " , Accuracy)

# full weight optimization section

In [102]:
m1 = load_model(model_path + '/resnet152.h5')

m1 = Model(inputs = m1.inputs, outputs = m1.outputs,
          name = 'resnet152')


m2 = load_model(model_path + '/dense201.h5')
m2 = Model(inputs = m2.inputs, outputs = m2.outputs,
          name = 'densenet201')


m3 = load_model(model_path + '/vgg16.h5')
m3 = Model(inputs = m3.inputs, outputs = m3.outputs,
          name = 'vgg16')

models = [m1, m2, m3]

In [103]:
len(models)

3

In [104]:
model_input = tf.keras.Input(shape = (img_height,img_width,3))
model_outputs = [model(model_input) for model in models]

In [107]:
class WeightedAverageLayer(tf.keras.layers.Layer):
    def __init__(self, w1, w2, w3, **kwargs):
        super(WeightedAverageLayer, self).__init__(**kwargs)
        self.w1 = w1
        self.w2 = w2
        self.w3 = w3

    def call(self, inputs):
        return self.w1 * inputs[0] + self.w2 * inputs[1] + self.w3 * inputs[2]
    #added -------------------------------------------------------------------------------------------------------------
    # def get_config(self, w1, w2, w3):
    #     config = super().get_config()
    #     config.update({
    #         self.w1 = w1
    #         self.w2 = w2
    #         self.w3 = w3
    #         })
    #     return config

In [108]:
num = [[.5, .1, .4]]

In [ ]:

opt = Adam(learning_rate = 0.0001)
j = 0
for i in num:
    a = num[j][0]
    b = num[j][1]
    c = num[j][2]
    print(a,b,c)
    ensemble_output = WeightedAverageLayer(a,b,c)(model_outputs)
    final_model = tf.keras.Model(inputs = model_input, outputs = ensemble_output)

    final_model.compile(optimizer = opt, loss=tf.keras.losses.BinaryCrossentropy(), metrics=['accuracy'])

    start_time = time.time()

    history_final = final_model.fit(
        train_generator,
          steps_per_epoch=train_generator.n // batch_size,
          epochs=50,
          validation_data=valid_generator,
          validation_steps=valid_generator.n // batch_size
    )

    end_time = time.time()
    training_time = end_time - start_time

    print(f"Training time: {training_time} seconds")

    loss,acc = final_model.evaluate(valid_generator)

    # with open('/content/drive/MyDrive/Colab Notebooks/model/original data model/weight(original).txt', 'a') as file:
    # #for number in numbers:
    #     # Write the number to the file, followed by a newline
    #     file.write(str(a) + ' ' + str(b) + ' ' + str(c) + ' ' + str(loss) + ' ' + str(acc) + '\n')

    j = j+1

In [ ]:
final_model.save(model_path + '/final.h5')

# Model evaluation

In [ ]:
# Evaluate the model
valid_generator.reset()
start_time = time.time()
#predictions = model.predict(valid_generator, steps=valid_generator.n // batch_size)
predictions = final_model.predict(valid_generator)
end_time = time.time()

inference_time_batch = end_time - start_time
print("Inference time:", inference_time_batch)

In [ ]:
inference_time_batch

In [ ]:
# Get the predicted classes
predicted_classes = np.argmax(predictions, axis=1)

# Get the true classes
true_classes = valid_generator.classes

In [ ]:
# Calculate performance metrics
confusion = confusion_matrix(true_classes, predicted_classes)
tn, fp, fn, tp = confusion.ravel()
specificity = tn / (tn + fp)
precision = precision_score(true_classes, predicted_classes)
recall = recall_score(true_classes, predicted_classes)
f1 = f1_score(true_classes, predicted_classes)

# Print the performance metrics
print('Specificity:', specificity)
print('Precision:', precision)
print('Recall:', recall)
print('F1 Score:', f1)

In [ ]:
Accuracy =  (tp + tn) / ( tp + tn + fp + fn)
print(Accuracy)

In [ ]:
loss,acc = final_model.evaluate(valid_generator)

In [ ]:
# Plot the confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(confusion, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.savefig(model_path + '/final_conf_matrix.png')
plt.show()

In [ ]:
#plot loss and accuracy graph

x = np.arange(1, 20)
y = np.arange(0.0, 1.0)
#y = np.array([0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0])

#Plot the Loss Curves
plt.figure(figsize=[6,4])
#plt.xlim(0, 20)
#plt.ylim(0, 1)
#plt.xticks(np.arange(2,21,2))
#plt.yticks(np.arange(0.0, 1.0, 0.1))
print(len(history_final.history['loss']) )
plt.plot(range(1, len(history_final.history['loss']) + 1 ), history_final.history['loss'],'r')
plt.plot(range(1, len(history_final.history['val_loss']) + 1), history_final.history['val_loss'],'b')
#plt.plot(history_final.history['loss'],'r')
#plt.plot(history_final.history['val_loss'],'b')
plt.legend(['Training loss', 'Validation Loss'])
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Ensemble Model Loss Curves')
plt.xticks([10, 20])
plt.yticks([.2, .4, .6, .8, 1, 1.2])
#plt.show()
plt.savefig(model_path + '/final_loss.png')

#Plot the Accuracy Curves
plt.figure(figsize=[6,4])
#plt.plot(history_final.history['accuracy'],'r')
#plt.plot(history_final.history['val_accuracy'],'b')
plt.plot(range(1, len(history_final.history['accuracy']) + 1 ), history_final.history['accuracy'],'r')
plt.plot(range(1, len(history_final.history['val_accuracy']) + 1), history_final.history['val_accuracy'],'b')
plt.legend(['Training Accuracy', 'Validation Accuracy'])
plt.xlabel('Epochs ')
plt.ylabel('Accuracy')
plt.title('Ensemble Model Accuracy Curves')
plt.xticks([10, 20])
plt.yticks([.2, .4, .6, .8, 1, 1.2])
#plt.show()
plt.savefig(model_path + '/final_acc.png')

# Model Evaluation Metrics
[link](https://machinelearningmastery.com/how-to-calculate-precision-recall-f1-and-more-for-deep-learning-models/)

In [ ]:
# Evaluate the model
valid_generator.reset()
#predictions = model.predict(valid_generator, steps=valid_generator.n // batch_size)
predictions = final_model.predict(valid_generator)   #yhat_probs

# Get the predicted classes
predicted_classes = np.argmax(predictions, axis=1)    #yhat_classes

# Get the true classes
true_classes = valid_generator.classes


# Calculate performance metrics
confusion = confusion_matrix(true_classes, predicted_classes)
tn, fp, fn, tp = confusion.ravel()
print("tn: ", tn ,"fp: ", fp,"fn: ", fn,"tp: ", tp)
specificity = tn / (tn + fp)
precision = precision_score(true_classes, predicted_classes)
recall = recall_score(true_classes, predicted_classes)
f1 = f1_score(true_classes, predicted_classes)

# Print the performance metrics
print('Specificity:', specificity)
print('Precision:', precision)
print('Recall:', recall)
print('F1 Score:', f1)

Accuracy =  (tp + tn) / ( tp + tn + fp + fn)
print("Accuracy: " , Accuracy)